In [2]:
!pip install jieba

    100% |████████████████████████████████| 19.2MB 763kB/s ta 0:00:011
  Running setup.py bdist_wheel for jieba ... done
  Stored in directory: /Users/yangkaixiang/Library/Caches/pip/wheels/af/e4/8e/5fdd61a6b45032936b8f9ae2044ab33e61577950ce8e0dec29
Successfully built jieba


In [4]:
#加载所需的包
import jieba  
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans 
from collections import defaultdict
from matplotlib import pyplot as plt
from sklearn.preprocessing import Normalizer # 把tfidf归一化为单位向量
from sklearn.naive_bayes import MultinomialNB # 多项式贝叶斯模型
from sklearn.model_selection import train_test_split # 拆分数据集
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer # 统计词频、计算tfidf
from sklearn.metrics import classification_report # 分类报告 评估模型

In [6]:
#读取数据探查数据的记录与字段数，查看前五行数据
df = pd.read_csv('sqlResult.csv', encoding='gb18030')
print(df.shape)
print(df.head())

(89611, 7)
      id      author                  source  \
0  89617         NaN  快科技@http://www.kkj.cn/   
1  89616         NaN  快科技@http://www.kkj.cn/   
2  89615         NaN  快科技@http://www.kkj.cn/   
3  89614         NaN                     新华社   
4  89613  胡淑丽_MN7479                   深圳大件事   

                                             content  \
0  此外，自本周（6月12日）起，除小米手机6等15款机型外，其余机型已暂停更新发布（含开发版/...   
1  骁龙835作为唯一通过Windows 10桌面平台认证的ARM处理器，高通强调，不会因为只考...   
2  此前的一加3T搭载的是3400mAh电池，DashCharge快充规格为5V/4A。\r\n...   
3    这是6月18日在葡萄牙中部大佩德罗冈地区拍摄的被森林大火烧毁的汽车。新华社记者张立云摄\r\n   
4  （原标题：44岁女子跑深圳约会网友被拒，暴雨中裸身奔走……）\r\n@深圳交警微博称：昨日清...   

                                             feature  \
0  {"type":"科技","site":"cnbeta","commentNum":"37"...   
1  {"type":"科技","site":"cnbeta","commentNum":"15"...   
2  {"type":"科技","site":"cnbeta","commentNum":"18"...   
3  {"type":"国际新闻","site":"环球","commentNum":"0","j...   
4  {"type":"新闻","site":"网易热门","commentNum":"978",...   

                          

In [7]:
#删掉content列内容为空的记录并查看剩下的数据量89611-87054=5257
df = df.dropna(subset=['content'])
print(df.shape)

(87054, 7)


In [8]:
with open('chinese_stopwords.txt', 'r', encoding='utf-8') as file:
    # 直接file.readlines()得到的元素最后是换行符，用切片去掉后面的换行符
    stopwords = [i[:-1] for i in file.readlines()]
    
def split_text(text):
    text = text.replace(" ","").replace("\n",'')
    text2 = jieba.cut(text)
    result = " ".join([x for x in text2 if x not in stopwords])
    return result

In [12]:
stopwords[:10]

['，', '的', '。', '、', '在', '了', '是', '\u3000', '“', '”']

In [10]:
print(split_text(df.iloc[0].content))

Building prefix dict from the default dictionary ...
Dumping model to file cache /var/folders/gj/2mf5qmrs21ngcz96t_z2316r0000gn/T/jieba.cache
Loading model cost 1.232 seconds.
Prefix dict has been built successfully.


 当然 MIUI9 确切 信息 等待 官方消息 


In [20]:
corpus = np.array(list(map(split_text,[str(i) for i in df.content])))
corpus

array(['此外 本周 除 小米 手机 款 机型 外 机型 暂停 更新 发布 含 开发 版 体验版 内测 稳定版 暂不受 影响 确保 工程师 集中 全部 精力 进行 系统优化 工作 有人 猜测 精力 主要 用到 MIUI9 研发 之中 \r MIUI8 去年 发布 距今已有 一年 有余 更新换代 \r 当然 MIUI9 确切 信息 等待 官方消息 \r',
       '骁龙 835 唯一 Windows10 桌面 平台 认证 ARM 处理器 高通 强调 不会 只 考虑 性能 屏蔽掉 小 核心 相反 正 联手 微软 找到 一种 适合 桌面 平台 兼顾 性能 功耗 完美 方案 \r 报道 微软 已经 拿到 一些 源码 Windows10 更好 理解 big little 架构 \r 资料 显示 骁龙 835 一款 集成 CPU GPU 基带 蓝牙 Wi Fi SoC 传统 Wintel 方案 节省 至少 30% PCB 空间 \r 按计划 今年 Q4 华硕 惠普 联想 首发 骁龙 835Win10 电脑 预计 均 二合一 形态 产品 \r 当然 高通 骁龙 未来 也许 见到 三星 Exynos 联发科 华为 麒麟 小米 澎湃 进入 Windows10 桌面 平台 \r',
       '此前 一加 3T 搭载 3400mAh 电池 DashCharge 快充 规格 5V 4A \r 电池 缩水 可能 刘作 虎 所说 一加 手机 市面 轻薄 大屏 旗舰 设定 \r 按照 目前 掌握 资料 一加 手机 拥有 5.5 寸 1080P 三星 AMOLED 显示屏 6G 8GBRAM 64GB 128GBROM 双 1600 万 摄像头 备货 量 惊喜 \r 京东 泄露 信息 一加 售价 xx99 应该 2799 2899 2999 \r',
       ...,
       '\\ n \\ n2017 29 法国巴黎 郊外 凡尔赛宫 法国 总统 马克 龙 出席 新闻 发布会 新华 法新 \\ n \\ n 新华社 北京 日电 记者 张 旌 法国 定于 举行 议会选举 首轮 投票 最新 民调 结果显示 总统 埃马 纽埃 尔 马克 龙 领导 共和国 前进 运动 这场 选举 获得 压倒性 胜利 不仅 赢得 议会 多数 席位 数量 可望 创下 1968 以来 历届 执政

In [21]:
corpus.shape

(87054,)

In [26]:
countervectorizer = CountVectorizer(encoding='gb18030',min_df=0.015)
countvector = countervectorizer.fit_transform(corpus)

tfidftransformer = TfidfTransformer()
tfidf = tfidftransformer.fit_transform(countvector)
print(tfidf.shape)

(87054, 884)


In [28]:
label = list(map(lambda source: 1 if '新华社' in str(source) else 0,df.source))

In [29]:
x_train, x_test, y_train, y_test = train_test_split(tfidf.toarray(), label, test_size=0.3)

In [35]:
x_train.shape, x_test.shape, np.array(y_train).shape, np.array(y_test).shape

((60937, 884), (26117, 884), (60937,), (26117,))

In [36]:
#创建模型
estimator = MultinomialNB()
estimator

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [37]:
estimator.fit(x_train, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [43]:
y_pred = estimator.predict(x_test)

In [54]:
y_test = np.array(y_test)
print(classification_report(y_test, y_pred, labels=[0, 1], target_names=['others', '新华社']))

              precision    recall  f1-score   support

      others       0.43      0.65      0.52      2525
         新华社       0.96      0.91      0.93     23592

   micro avg       0.88      0.88      0.88     26117
   macro avg       0.69      0.78      0.72     26117
weighted avg       0.91      0.88      0.89     26117



In [55]:
prediction = estimator.predict(tfidf.toarray())
print(len(prediction))
prediction

87054


array([0, 0, 0, ..., 1, 0, 1])

In [56]:
len(label)

87054

In [52]:
compare_news_df = pd.DataFrame({"prediction": prediction, 'labels': label})
compare_news_df.head(3)

,prediction,labels
0,0,0
1,0,0
2,0,0


In [60]:
# 在实际非新华社的文章中找出预测为新华社风格的文章
compare_news_df_new_index = compare_news_df[(compare_news_df['labels'] == 0) & (compare_news_df['prediction'] == 1)]
print(compare_news_df_new_index.shape)
compare_news_df_new_index.head(2)

(2825, 2)


,prediction,labels
4,1,0
24,1,0


In [59]:
compare_news_df[compare_news_df['prediction'] == 1].index

Int64Index([    3,     4,    24,    25,    28,    30,    32,    35,    37,
               38,
            ...
            87042, 87043, 87044, 87045, 87046, 87047, 87048, 87049, 87051,
            87053],
           dtype='int64', length=74282)

In [58]:
# 实际不是新华社的新闻的 index
no_xinhua_news_index = compare_news_df[compare_news_df['labels'] == 0].index
no_xinhua_news_index

Int64Index([   0,    1,    2,    4,    5,    6,    7,    8,    9,   10,
            ...
            8546, 8547, 8548, 8549, 8550, 8551, 8552, 8553, 8554, 8555],
           dtype='int64', length=8380)

### 结论：8380条不是新华社发布的，其中的2825条新闻与新华社风格相似，属于疑似抄袭新华社的新闻